In [1]:
import os
import numpy as np
import torch
import torch.utils.data
from sklearn.metrics import mean_absolute_error
from sklearn.metrics import mean_squared_error
from scipy.sparse.linalg import eigs

###### Function for Computing Scaled Laplacian

In [2]:
def scaled_Laplacian(W):
    '''
    compute \tilde{L}

    Parameters
    ----------
    W: np.ndarray, shape is (N, N), N is the num of vertices

    Returns
    ----------
    scaled_Laplacian: np.ndarray, shape (N, N)

    '''
    ###Checking if the number of rows and columns of an adjacenecy matrix
    ####are same or not
    assert W.shape[0] == W.shape[1]

    ### First sum each row of the adjacency matrix, and we obtain the degress of each vertex in that row
    ### Secondly, a diagonal matrix has been created with degree of each vertex in the diagonal
    #### Finally, 'D' is the sparse matrix containing only degrees of each vertex
    D = np.diag(np.sum(W, axis=1))

    #### 'l' is the unormalized Laplacian Matrix obtained by subtraction
    ### of Adjacenecy Matrix from the Diagonal Matrix
    L = D - W

    #### First of all from the Laplacian Matrix, with the help of 'eigs' function largest value for eigen value and eigne vector
    ##### has been evaluated
    #### Secondly, eigen value has been only kept
    #### Thirdly, eigen value beign a complex number, only real part is kept and saved as lamda_max
    lambda_max = eigs(L, k=1, which='LR')[0].real

    #### Finally Scaled Laplacian Matrix value is Obtained
    return (2 * L) / lambda_max - np.identity(W.shape[0])


##### Function for Computing Chebyshev Polynomials

In [3]:
def cheb_polynomial(L_tilde, K):
    '''
    compute a list of chebyshev polynomials from T_0 to T_{K-1}

    Parameters
    ----------
    L_tilde: scaled Laplacian, np.ndarray, shape (N, N)

    K: the maximum order of chebyshev polynomials

    Returns
    ----------
    cheb_polynomials: list(np.ndarray), length: K, from T_0 to T_{K-1}

    '''
    ### The value of N is set to number of rows of a Laplacian Matric
    N = L_tilde.shape[0]

    ### cheb_polynimials conatins the zeroth order Chebyshev Polynomial T(0) = np.identity(N)
    ### and the first order Chebyshev Polynomial T(1) = L_tilde.copy()
    cheb_polynomials = [np.identity(N), L_tilde.copy()]

    ### The loop computes the next higher order of Chebyshev Polynomials form order 2 to order k
    ### The next order recurrence Chebysehev Polynomial is given by Tk(x) =2xTk-1(x)-Tk-2(x)
    ## Since the loop starts from order 2, I am experimenting with values K = 3, 4, 5
    for i in range(2, K):
        cheb_polynomials.append(2 * L_tilde * cheb_polynomials[i - 1] - cheb_polynomials[i - 2])

    return cheb_polynomials

In [4]:
import torch
import torch.nn as nn
import torch.nn.functional as F

###### Defining the Spatial Attention Layer

In [5]:
class Spatial_Attention_layer(nn.Module):
    '''
    compute spatial attention scores
    : in_channels: number of features of a vertex, here for PEMS07, it's 1
    : num_of_timesteps represents number of timesegments in the input data
    '''
    def __init__(self, DEVICE, in_channels, num_of_vertices, num_of_timesteps):
        super(Spatial_Attention_layer, self).__init__()
        self.W1 = nn.Parameter(torch.FloatTensor(num_of_timesteps).to(DEVICE))
        self.W2 = nn.Parameter(torch.FloatTensor(in_channels, num_of_timesteps).to(DEVICE))
        self.W3 = nn.Parameter(torch.FloatTensor(in_channels).to(DEVICE))
        self.bs = nn.Parameter(torch.FloatTensor(1, num_of_vertices, num_of_vertices).to(DEVICE))
        self.Vs = nn.Parameter(torch.FloatTensor(num_of_vertices, num_of_vertices).to(DEVICE))


    def forward(self, x):
        '''
        :param x: (batch_size, N, F_in, T)
        :return: (B,N,N)
        '''

        lhs = torch.matmul(torch.matmul(x, self.W1), self.W2)  # (b,N,F,T)(T)->(b,N,F)(F,T)->(b,N,T)

        rhs = torch.matmul(self.W3, x).transpose(-1, -2)  # (F)(b,N,F,T)->(b,N,T)->(b,T,N)

        product = torch.matmul(lhs, rhs)  # (b,N,T)(b,T,N) -> (B, N, N)

        S = torch.matmul(self.Vs, torch.sigmoid(product + self.bs))  # (N,N)(B, N, N)->(B,N,N)

        ##Normalized Spatial Attention scores observes the spatial dependency between the vertices
        S_normalized = F.softmax(S, dim=1)

        return S_normalized

###### Chebyshev's Convolution with Spatial Attention
###### It is alsp known as Graph Convolution with Spatial Attention

In [6]:
class cheb_conv_withSAt(nn.Module):
    '''
    K-order chebyshev graph convolution
    '''

    def __init__(self, K, cheb_polynomials, in_channels, out_channels):
        '''
        :param K: int
        :param in_channles: int, num of channels in the input sequence
        :param out_channels: int, num of channels in the output sequence
        '''
        super(cheb_conv_withSAt, self).__init__()
        self.K = K
        self.cheb_polynomials = cheb_polynomials
        self.in_channels = in_channels
        self.out_channels = out_channels
        self.DEVICE = cheb_polynomials[0].device

        ### Theta consists of a list of learnable weight matrices of each of shape (in_channels x out_channels) for each of the
        #### Chebyshev Polynomial, where number of Chebyshev is represneted by 'K'
        self.Theta = nn.ParameterList([nn.Parameter(torch.FloatTensor(in_channels, out_channels).to(self.DEVICE)) for _ in range(K)])

    def forward(self, x, spatial_attention):
        '''
        Chebyshev graph convolution operation
        :param x: (batch_size, N, F_in, T)
        :return: (batch_size, N, F_out, T)
        '''
        #### Getting shape of the input Graph signal
        batch_size, num_of_vertices, in_channels, num_of_timesteps = x.shape

        outputs = []

        for time_step in range(num_of_timesteps):
            ####getting the data for a certain time segment
            graph_signal = x[:, :, :, time_step]  # (b, N, F_in)

            output = torch.zeros(batch_size, num_of_vertices, self.out_channels).to(self.DEVICE)  # (b, N, F_out)
            #### Looping Over Chebyshev Polynomial Order
            #### In order to Perform Graph Convolution Operation with Spatial Attention
            for k in range(self.K):

                T_k = self.cheb_polynomials[k]  # (N,N)

                ###Hadamarad Product between Chebyshev and Spatial Attention
                T_k_with_at = T_k.mul(spatial_attention)   # (N,N)*(N,N) = (N,N)

                theta_k = self.Theta[k]  # (in_channel, out_channel)

                rhs = T_k_with_at.permute(0, 2, 1).matmul(graph_signal)  # (N, N)(b, N, F_in) = (b, N, F_in)

                output = output + rhs.matmul(theta_k)  # (b, N, F_in)(F_in, F_out) = (b, N, F_out)
            ### Additional dimension is added in order to understand that Graph Concolution
            #### for a particular timestamp/timesegment has been done
            outputs.append(output.unsqueeze(-1))  # (b, N, F_out, 1)
        ####Concatenating the results accross all timesteps and applying Rely activation function
        #### on each one of them
        return F.relu(torch.cat(outputs, dim=-1))  # (b, N, F_out, T)


###### Temporal Attention Layer

In [7]:
class Temporal_Attention_layer(nn.Module):
    def __init__(self, DEVICE, in_channels, num_of_vertices, num_of_timesteps):
        super(Temporal_Attention_layer, self).__init__()
        self.U1 = nn.Parameter(torch.FloatTensor(num_of_vertices).to(DEVICE))
        self.U2 = nn.Parameter(torch.FloatTensor(in_channels, num_of_vertices).to(DEVICE))
        self.U3 = nn.Parameter(torch.FloatTensor(in_channels).to(DEVICE))
        self.be = nn.Parameter(torch.FloatTensor(1, num_of_timesteps, num_of_timesteps).to(DEVICE))
        self.Ve = nn.Parameter(torch.FloatTensor(num_of_timesteps, num_of_timesteps).to(DEVICE))

    def forward(self, x):
        '''
        :param x: (batch_size, N, F_in, T)
        :return: (B, T, T)
        '''
        _, num_of_vertices, num_of_features, num_of_timesteps = x.shape

        lhs = torch.matmul(torch.matmul(x.permute(0, 3, 2, 1), self.U1), self.U2)
        # x:(B, N, F_in, T) -> (B, T, F_in, N)
        # (B, T, F_in, N)(N) -> (B,T,F_in)
        # (B,T,F_in)(F_in,N)->(B,T,N)

        rhs = torch.matmul(self.U3, x)  # (F)(B,N,F,T)->(B, N, T)

        product = torch.matmul(lhs, rhs)  # (B,T,N)(B,N,T)->(B,T,T)

        E = torch.matmul(self.Ve, torch.sigmoid(product + self.be))  # (B, T, T)

        E_normalized = F.softmax(E, dim=1)
        ### Temporal Attention score measures the importance of each timesteps fore a node in the Graph
        return E_normalized


###### Describing a Singular ASTGCN Block

In [8]:
class ASTGCN_block(nn.Module):

    def __init__(self, DEVICE, in_channels, K, nb_chev_filter, nb_time_filter, time_strides, cheb_polynomials, num_of_vertices, num_of_timesteps):
        super(ASTGCN_block, self).__init__()
        self.TAt = Temporal_Attention_layer(DEVICE, in_channels, num_of_vertices, num_of_timesteps)
        self.SAt = Spatial_Attention_layer(DEVICE, in_channels, num_of_vertices, num_of_timesteps)
        self.cheb_conv_SAt = cheb_conv_withSAt(K, cheb_polynomials, in_channels, nb_chev_filter)

        ###Applies convolution along temporal dimension
        ##nb_chev_filter relates to order of Chebyshev Polynmials, representing number of chebyshev outputs coming
        ### nb_time_filter relates to the number of timesteps for each of the chebyshev output
        #kernel size depends on the value of K, as mentioned by author depending on the temporal dimension
        ## of neighbouring vertices
        self.time_conv = nn.Conv2d(nb_chev_filter, nb_time_filter, kernel_size=(1, 3), stride=(1, time_strides), padding=(0, 1))

        ####(1x1) convolution is applied for residual connections
        self.residual_conv = nn.Conv2d(in_channels, nb_time_filter, kernel_size=(1, 1), stride=(1, time_strides))

        ####Normalization Layer
        self.ln = nn.LayerNorm(nb_time_filter)

    def forward(self, x):
        '''
        :param x: (batch_size, N, F_in, T)
        :return: (batch_size, N, nb_time_filter, T)
        '''
        batch_size, num_of_vertices, num_of_features, num_of_timesteps = x.shape

        # Step 1: Temporal attention is computed
        temporal_At = self.TAt(x)  # (b, T, T)

        # Step 2: Temporal attention is applied on the input because input consists of
        # graph signal information of different timesteps
        # Temporal attention is applied to focus on temporal correlations between neighbouring timestep data
        ## of a node
        x_TAt = torch.matmul(x.reshape(batch_size, -1, num_of_timesteps), temporal_At).reshape(batch_size, num_of_vertices, num_of_features, num_of_timesteps)

        # Step 3: Spatial Attention is applied on the temporal attention layer output
        spatial_At = self.SAt(x_TAt)

        # Step 4: Graph Convolution is applied using the Chebyshev Polynlomials of order K on the graph signal and the
        # spatial attention layer output, in order to focus on the sptial correlations of (K-1) neighbouring nodes
        ## of a certain node in the Graph
        spatial_gcn = self.cheb_conv_SAt(x, spatial_At)  # (b,N,F,T)
        # spatial_gcn = self.cheb_conv(x)

        # convolution along the time axis
        time_conv_output = self.time_conv(spatial_gcn.permute(0, 2, 1, 3))  # (b,N,F,T)->(b,F,N,T) (1,3)->(b,F,N,T)

        #print("Shape of x_residual:", x_residual.shape)
        #print("Shape of time_conv_output:", time_conv_output.shape)

        # residual shortcut is applied directly on the input signal with the help of a convolution layer
        x_residual = self.residual_conv(x.permute(0, 2, 1, 3))  # (b,N,F,T)->(b,F,N,T) (1,1)->(b,F,N,T)

        #print("Shape of x_residual:", x_residual.shape)
        #print("Shape of time_conv_output:", time_conv_output.shape)

        ### Finally, the residual output is equal to the summation of the residual shortcut output and the final
        ### output coming from the time-axis layer
        x_residual = self.ln(F.relu(x_residual + time_conv_output).permute(0, 3, 2, 1)).permute(0, 2, 3, 1)
        # (b,F,N,T)->(b,T,N,F) -ln-> (b,T,N,F)->(b,N,F,T)

        return x_residual

###### Describing a series of ASTGCN Blocks inside the submodule Class

In [9]:
class ASTGCN_submodule(nn.Module):

    def __init__(self, DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter, time_strides, cheb_polynomials, num_for_predict, len_input, num_of_vertices):
        '''
        :param nb_block:
        :param in_channels:
        :param K:
        :param nb_chev_filter:
        :param nb_time_filter:
        :param time_strides:
        :param cheb_polynomials:
        :param nb_predict_step:
        '''

        super(ASTGCN_submodule, self).__init__()

        self.BlockList = nn.ModuleList([ASTGCN_block(DEVICE, in_channels, K, nb_chev_filter, nb_time_filter, time_strides, cheb_polynomials, num_of_vertices, len_input)])

        self.BlockList.extend([ASTGCN_block(DEVICE, nb_time_filter, K, nb_chev_filter, nb_time_filter, 1, cheb_polynomials, num_of_vertices, len_input//time_strides) for _ in range(nb_block-1)])

        ###Adding a Final Convolution Layer after the output from the final ASTGCN Block
        self.final_conv = nn.Conv2d(int(len_input/time_strides), num_for_predict, kernel_size=(1, nb_time_filter))

        self.DEVICE = DEVICE

        self.to(DEVICE)

    def forward(self, x):
        '''
        :param x: (B, N_nodes, F_in, T_in)
        :return: (B, N_nodes, T_out)
        '''
        for block in self.BlockList:
            x = block(x)

        output = self.final_conv(x.permute(0, 3, 1, 2))[:, :, :, -1].permute(0, 2, 1)
        # (b,N,F,T)->(b,T,N,F)-conv<1,F>->(b,c_out*T,N,1)->(b,c_out*T,N)->(b,N,T)

        return output


###### Making Model Function

In [10]:
def make_model(DEVICE, nb_block, in_channels, K, nb_chev_filter, nb_time_filter,
               time_strides, adj_mx, num_for_predict, len_input, num_of_vertices):
    '''

    :param DEVICE:
    :param nb_block:
    :param in_channels:
    :param K:
    :param nb_chev_filter:
    :param nb_time_filter:
    :param time_strides:
    :param cheb_polynomials:
    :param nb_predict_step:
    :param len_input
    :return:
    '''
    L_tilde = scaled_Laplacian(adj_mx)
    cheb_polynomials = [torch.from_numpy(i).type(torch.FloatTensor).to(DEVICE) for i in cheb_polynomial(L_tilde, K)]
    model = ASTGCN_submodule(DEVICE, nb_block, in_channels, K,
                             nb_chev_filter, nb_time_filter, time_strides, cheb_polynomials,
                             num_for_predict, len_input, num_of_vertices)

    ###Applying Xavier Initialization to parameters having diemnsion > 1
    for p in model.parameters():
        if p.dim() > 1:
            nn.init.xavier_uniform_(p)
        else:
            nn.init.uniform_(p)

    return model

In [11]:
import torch

if torch.cuda.is_available():
    DEVICE = torch.device("cuda")
else:
    DEVICE = torch.device("cpu")

In [12]:
!pip install torch-geometric

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 64.2/64.2 kB 1.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 17.1 MB/s eta 0:00:00


In [13]:
!pip install tensorboardX

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.7/101.7 kB 2.9 MB/s eta 0:00:00


In [14]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [15]:
import os
from time import time
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx



import torch
import torch.nn as nn
import torch.optim as optim
import torch.nn.functional as F
from scipy.sparse.linalg import eigs


USE_CUDA = torch.cuda.is_available()
DEVICE = torch.device('cuda:0')
print("CUDA:", USE_CUDA, DEVICE)

from tensorboardX import SummaryWriter
sw = SummaryWriter(logdir='.', flush_secs=5)

import math
from typing import Optional, List, Union

import torch
import torch.nn as nn
from torch.nn import Parameter
import torch.nn.functional as F

from torch_geometric.data import Data
from torch_geometric.typing import OptTensor
from torch_geometric.nn.conv import MessagePassing
from torch_geometric.transforms import LaplacianLambdaMax
from torch_geometric.utils import remove_self_loops, add_self_loops, get_laplacian
from torch_geometric.utils import to_dense_adj
#from torch_scatter import scatter_add

CUDA: True cuda:0


###### Loading the Dataset and dividing the Dataset into Training, Testing and Validation

In [16]:
def load_graphdata_channel1(batch_size,shuffle=True, DEVICE = torch.device('cuda:0')):
    '''
    :param DEVICE:
    :param batch_size: int
    :return:
    three DataLoaders, each dataloader contains:
    test_x_tensor: (B, N_nodes, in_feature, T_input)
    test_decoder_input_tensor: (B, N_nodes, T_output)
    test_target_tensor: (B, N_nodes, T_output)
    '''

    #file = os.path.basename(graph_signal_matrix_filename).split('.')[0]
    #filename = os.path.join('../input/processing-traffic-data-for-deep-learning-projects/', file + '_r' + str(num_of_hours) + '_d' + str(num_of_days) + '_w' + str(num_of_weeks)) +'_astcgn'
    #print('load file:', filename)

    file_data = np.load(r"/content/drive/MyDrive/COMP9491_ASTGCN_Model/Dataset_PEMS07/PEMS04_304r4_304d0_304w0_astcgn.npz")
    train_x = file_data['train_x']  # (10181, 307, 1, 12)
    train_x = train_x[:, :, 0:1, :]
    train_target = file_data['train_target']  # (10181, 307, 12)

    val_x = file_data['val_x']
    val_x = val_x[:, :, 0:1, :]
    val_target = file_data['val_target']

    test_x = file_data['test_x']
    test_x = test_x[:, :, 0:1, :]
    test_target = file_data['test_target']

    mean = file_data['mean'][:, :, 0:1, :]  # (1, 1, 3, 1)
    std = file_data['std'][:, :, 0:1, :]  # (1, 1, 3, 1)

    # ------- train_loader -------
    train_x_tensor = torch.from_numpy(train_x).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, T)
    train_target_tensor = torch.from_numpy(train_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)
    train_dataset = torch.utils.data.TensorDataset(train_x_tensor, train_target_tensor)
    train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=batch_size, shuffle=shuffle)

    # ------- val_loader -------
    val_x_tensor = torch.from_numpy(val_x).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, T)
    val_target_tensor = torch.from_numpy(val_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)
    val_dataset = torch.utils.data.TensorDataset(val_x_tensor, val_target_tensor)
    val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=batch_size, shuffle=False)

    # ------- test_loader -------
    test_x_tensor = torch.from_numpy(test_x).type(torch.FloatTensor).to(DEVICE)  # (B, N, F, T)
    test_target_tensor = torch.from_numpy(test_target).type(torch.FloatTensor).to(DEVICE)  # (B, N, T)
    test_dataset = torch.utils.data.TensorDataset(test_x_tensor, test_target_tensor)
    test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=batch_size, shuffle=False)

    # print
    print('train:', train_x_tensor.size(), train_target_tensor.size())
    print('val:', val_x_tensor.size(), val_target_tensor.size())
    print('test:', test_x_tensor.size(), test_target_tensor.size())

    return train_loader, train_target_tensor, val_loader, val_target_tensor, test_loader, test_target_tensor, mean, std

###### Loading the Data and receiving the dimensions of training, testing and validation tensors

In [17]:
batch_size = 64

train_loader, train_target_tensor, val_loader, val_target_tensor, test_loader, test_target_tensor, _mean, _std = load_graphdata_channel1(batch_size)

train: torch.Size([10186, 307, 1, 12]) torch.Size([10186, 307, 3])
val: torch.Size([3396, 307, 1, 12]) torch.Size([3396, 307, 3])
test: torch.Size([3396, 307, 1, 12]) torch.Size([3396, 307, 3])


###### Function To Obtain Adjacenecy Matrix from the Graph Data

In [18]:
def get_adjacency_matrix(distance_df_filename, num_of_vertices, id_filename=None):
    '''
    Parameters
    ----------
    distance_df_filename: str, path of the csv file contains edges information
    num_of_vertices: int, the number of vertices
    Returns
    ----------
    A: np.ndarray, adjacency matrix
    '''
    if 'npy' in distance_df_filename:  # false
        adj_mx = np.load(distance_df_filename)
        return adj_mx, None
    else:

        #--------------------------------------------- read from here
        import csv
        A = np.zeros((int(num_of_vertices), int(num_of_vertices)),dtype=np.float32)
        distaneA = np.zeros((int(num_of_vertices), int(num_of_vertices)), dtype=np.float32)

        #------------ Ignore
        if id_filename: # false
            with open(id_filename, 'r') as f:
                id_dict = {int(i): idx for idx, i in enumerate(f.read().strip().split('\n'))}

            with open(distance_df_filename, 'r') as f:
                f.readline()
                reader = csv.reader(f)
                for row in reader:
                    if len(row) != 3:
                        continue
                    i, j, distance = int(row[0]), int(row[1]), float(row[2])
                    A[id_dict[i], id_dict[j]] = 1
                    distaneA[id_dict[i], id_dict[j]] = distance
            return A, distaneA

        else:
         #-------------Continue reading
            with open(distance_df_filename, 'r') as f:
                f.readline()
                reader = csv.reader(f)
                for row in reader:
                    if len(row) != 3:
                        continue
                    i, j, distance = int(row[0]), int(row[1]), float(row[2])
                    A[i, j] = 1
                    distaneA[i, j] = distance
            return A, distaneA

In [19]:
id_filename = None
adj_filename = r'/content/drive/MyDrive/COMP9491_ASTGCN_Model/PEMS04_Recent_K_3_4_5/PEMS04.csv'
num_of_vertices = 307
adj_mx, distance_mx = get_adjacency_matrix(adj_filename, num_of_vertices, id_filename)

In [20]:
def masked_mape_np(y_true, y_pred, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(y_true)
        else:
            mask = np.not_equal(y_true, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mape = np.abs(np.divide(np.subtract(y_pred, y_true).astype('float32'),
                      y_true))
        mape = np.nan_to_num(mask * mape)
        return np.mean(mape)

In [21]:
def masked_mse(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        mask = ~torch.isnan(labels)
    else:
        mask = (labels != null_val)
    mask = mask.float()
    # print(mask.sum())
    # print(mask.shape[0]*mask.shape[1]*mask.shape[2])
    mask /= torch.mean((mask))
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = (preds - labels)**2
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)
    return torch.mean(loss)

In [22]:
def masked_rmse(preds, labels, null_val=np.nan):
    return torch.sqrt(masked_mse(preds=preds, labels=labels,
                                 null_val=null_val))

In [23]:
def masked_mae(preds, labels, null_val=np.nan):
    if np.isnan(null_val):
        ###creates a mask where values are present are set to True
        #### where missing values are present, are set to False
        mask = ~torch.isnan(labels)
    else:
        ### if there is no missing value, create a mask where values are False
        mask = (labels != null_val)
    mask = mask.float()

    ##normalizing the weight of the mask, by dividing with mean of mask values
    mask /= torch.mean((mask))

    ##Replaces any Missing value in Mask with Zero
    mask = torch.where(torch.isnan(mask), torch.zeros_like(mask), mask)
    loss = torch.abs(preds - labels)
    loss = loss * mask
    loss = torch.where(torch.isnan(loss), torch.zeros_like(loss), loss)

    ##Computing the meas value of mean absolute error
    return torch.mean(loss)

In [24]:
def masked_mae_test(y_true, y_pred, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(y_true)
        else:
            mask = np.not_equal(y_true, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mae = np.abs(np.subtract(y_pred, y_true).astype('float32'),
                      )
        mae = np.nan_to_num(mask * mae)
        return np.mean(mae)

In [25]:
def masked_rmse_test(y_true, y_pred, null_val=np.nan):
    with np.errstate(divide='ignore', invalid='ignore'):
        if np.isnan(null_val):
            mask = ~np.isnan(y_true)
        else:
            # null_val=null_val
            mask = np.not_equal(y_true, null_val)
        mask = mask.astype('float32')
        mask /= np.mean(mask)
        mse = ((y_pred- y_true)**2)
        mse = np.nan_to_num(mask * mse)
        return np.sqrt(np.mean(mse))

In [26]:
masked_flag=0
criterion = nn.L1Loss().to(DEVICE)
criterion_masked = masked_mae
loss_function = 'mse'

metric_method = 'unmask'
missing_value=0.0


if loss_function=='masked_mse':
    criterion_masked = masked_mse         #nn.MSELoss().to(DEVICE)
    masked_flag=1
elif loss_function=='masked_mae':
    criterion_masked = masked_mae
    masked_flag = 1
elif loss_function == 'mae':
    criterion = nn.L1Loss().to(DEVICE)
    ###indicating that standard loss function will be used
    masked_flag = 0
elif loss_function == 'rmse':
    criterion = nn.MSELoss().to(DEVICE)
    masked_flag= 0

In [27]:
model_hour_k4 = make_model(DEVICE, 2, 1, 4,
                        64, 64, 1, adj_mx,
                        3, 12, 307)

In [28]:
print(model_hour_k4)

ASTGCN_submodule(
  (BlockList): ModuleList(
    (0): ASTGCN_block(
      (TAt): Temporal_Attention_layer()
      (SAt): Spatial_Attention_layer()
      (cheb_conv_SAt): cheb_conv_withSAt(
        (Theta): ParameterList(
            (0): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
            (1): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
            (2): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
            (3): Parameter containing: [torch.float32 of size 1x64 (cuda:0)]
        )
      )
      (time_conv): Conv2d(64, 64, kernel_size=(1, 3), stride=(1, 1), padding=(0, 1))
      (residual_conv): Conv2d(1, 64, kernel_size=(1, 1), stride=(1, 1))
      (ln): LayerNorm((64,), eps=1e-05, elementwise_affine=True)
    )
    (1): ASTGCN_block(
      (TAt): Temporal_Attention_layer()
      (SAt): Spatial_Attention_layer()
      (cheb_conv_SAt): cheb_conv_withSAt(
        (Theta): ParameterList(
            (0): Parameter containing: [torch.flo

###### Defining Optimization and Config for Model, K = 4

In [29]:
learning_rate = 0.001
optimizer = optim.Adam(model_hour_k4.parameters(), lr=learning_rate)

In [30]:
print('Net\'s state_dict:')
total_param = 0
for param_tensor in model_hour_k4.state_dict():
    print(param_tensor, '\t', model_hour_k4.state_dict()[param_tensor].size(), '\t', model_hour_k4.state_dict()[param_tensor].device)
    total_param += np.prod(model_hour_k4.state_dict()[param_tensor].size())
print('Net\'s total params:', total_param)
#--------------------------------------------------
print('Optimizer\'s state_dict:')
for var_name in optimizer.state_dict():
    print(var_name, '\t', optimizer.state_dict()[var_name])

Net's state_dict:
BlockList.0.TAt.U1 	 torch.Size([307]) 	 cuda:0
BlockList.0.TAt.U2 	 torch.Size([1, 307]) 	 cuda:0
BlockList.0.TAt.U3 	 torch.Size([1]) 	 cuda:0
BlockList.0.TAt.be 	 torch.Size([1, 12, 12]) 	 cuda:0
BlockList.0.TAt.Ve 	 torch.Size([12, 12]) 	 cuda:0
BlockList.0.SAt.W1 	 torch.Size([12]) 	 cuda:0
BlockList.0.SAt.W2 	 torch.Size([1, 12]) 	 cuda:0
BlockList.0.SAt.W3 	 torch.Size([1]) 	 cuda:0
BlockList.0.SAt.bs 	 torch.Size([1, 307, 307]) 	 cuda:0
BlockList.0.SAt.Vs 	 torch.Size([307, 307]) 	 cuda:0
BlockList.0.cheb_conv_SAt.Theta.0 	 torch.Size([1, 64]) 	 cuda:0
BlockList.0.cheb_conv_SAt.Theta.1 	 torch.Size([1, 64]) 	 cuda:0
BlockList.0.cheb_conv_SAt.Theta.2 	 torch.Size([1, 64]) 	 cuda:0
BlockList.0.cheb_conv_SAt.Theta.3 	 torch.Size([1, 64]) 	 cuda:0
BlockList.0.time_conv.weight 	 torch.Size([64, 64, 1, 3]) 	 cuda:0
BlockList.0.time_conv.bias 	 torch.Size([64]) 	 cuda:0
BlockList.0.residual_conv.weight 	 torch.Size([64, 1, 1, 1]) 	 cuda:0
BlockList.0.residual_conv.bi

In [31]:
from tensorboardX import SummaryWriter
sw = SummaryWriter(logdir='.', flush_secs=5)

In [32]:
def compute_val_loss_astgcn(net, val_loader, criterion,  masked_flag,missing_value,sw, epoch, limit=None):
    '''
    for rnn, compute mean loss on validation set
    :param net: model
    :param val_loader: torch.utils.data.utils.DataLoader
    :param criterion: torch.nn.MSELoss
    :param sw: tensorboardX.SummaryWriter
    :param global_step: int, current global_step
    :param limit: int,
    :return: val_loss
    '''

    net.train(False)  # ensure dropout layers are in evaluation mode

    with torch.no_grad():

        val_loader_length = len(val_loader)  # nb of batch

        tmp = []  # batch loss

        for batch_index, batch_data in enumerate(val_loader):
            encoder_inputs, labels = batch_data
            outputs = net(encoder_inputs)
            if masked_flag:
                loss = criterion(outputs, labels, missing_value)
            else:
                loss = criterion(outputs, labels)

            tmp.append(loss.item())
            if batch_index % 100 == 0:
                print('validation batch %s / %s, loss: %.2f' % (batch_index + 1, val_loader_length, loss.item()))
            if (limit is not None) and batch_index >= limit:
                break

        validation_loss = sum(tmp) / len(tmp)
        sw.add_scalar('validation_loss', validation_loss, epoch)
    return validation_loss


##### Training for when K =4

In [33]:
global_step_k4 = 0
best_epoch_k4 = 0
best_val_loss_k4 = np.inf
start_time= time()

In [34]:
validation_loss_list_k4 = []
training_loss_list_k4 = []
final_train_loss_k4 = []

# train model
#masked_flag = 0
for epoch in range(20):

    params_filename = os.path.join('./', 'PEMS04_3pts_k4_epoch_%s.params' % epoch)
    masked_flag = 1

    if masked_flag:
        val_loss = compute_val_loss_astgcn(model_hour_k4, val_loader, criterion_masked, masked_flag,missing_value,sw, epoch)
    else:
        val_loss = compute_val_loss_astgcn(model_hour_k4, val_loader, criterion, masked_flag, missing_value, sw, epoch)

    ###appending the validation Loss in the List
    validation_loss_list_k4.append(val_loss)
    if val_loss < best_val_loss_k4:
        best_val_loss_k4 = val_loss
        best_epoch_k4 = epoch
        torch.save(model_hour_k4.state_dict(), params_filename)
        print('save parameters to file: %s' % params_filename)

    model_hour_k4.train()  # ensure dropout layers are in train mode

    for batch_index, batch_data in enumerate(train_loader):

        encoder_inputs, labels = batch_data

        optimizer.zero_grad()

        outputs = model_hour_k4(encoder_inputs)

        if masked_flag:
            loss = criterion_masked(outputs, labels,missing_value)
        else :
            loss = criterion(outputs, labels)


        loss.backward()

        optimizer.step()

        training_loss_k4 = loss.item()

        global_step_k4 += 1

        training_loss_list_k4.append(training_loss_k4)

        sw.add_scalar('training_loss', training_loss_k4, global_step_k4)
        #globalstep_training_loss_list.append(training_loss)

        if global_step_k4 % 200 == 0:

            print('global step: %s, training loss: %.2f, time: %.2fs' % (global_step_k4, training_loss_k4, time() - start_time))

    ##Estimating the Total Training Batch Loss
    train_batch_loss_k4 = sum(training_loss_list_k4)/len(training_loss_list_k4)
    final_train_loss_k4.append(train_batch_loss_k4)

print('best epoch:', best_epoch_k4)

validation batch 1 / 54, loss: 290.37
save parameters to file: ./PEMS04_3pts_k4_epoch_0.params
validation batch 1 / 54, loss: 151.37
save parameters to file: ./PEMS04_3pts_k4_epoch_1.params
global step: 200, training loss: 80.05, time: 95.42s
validation batch 1 / 54, loss: 53.56
save parameters to file: ./PEMS04_3pts_k4_epoch_2.params
global step: 400, training loss: 29.49, time: 175.32s
validation batch 1 / 54, loss: 33.04
save parameters to file: ./PEMS04_3pts_k4_epoch_3.params
global step: 600, training loss: 21.92, time: 255.15s
validation batch 1 / 54, loss: 28.05
save parameters to file: ./PEMS04_3pts_k4_epoch_4.params
global step: 800, training loss: 22.42, time: 334.58s
validation batch 1 / 54, loss: 26.85
save parameters to file: ./PEMS04_3pts_k4_epoch_5.params
validation batch 1 / 54, loss: 25.91
save parameters to file: ./PEMS04_3pts_k4_epoch_6.params
global step: 1000, training loss: 19.75, time: 420.77s
validation batch 1 / 54, loss: 25.46
save parameters to file: ./PEMS04

In [35]:
len(validation_loss_list_k4)

20

In [36]:
len(final_train_loss_k4)

20

In [37]:
validation_loss_list_k4

[219.9742317199707,
 110.4128160123472,
 43.47628722367463,
 26.869111290684454,
 23.037677305716056,
 21.91964221883703,
 21.12869946161906,
 20.701951415450484,
 20.28863181008233,
 20.345009220971,
 20.50826191019129,
 19.883104041770654,
 19.736917054211652,
 19.580256303151447,
 19.52108985406381,
 19.667629347907173,
 19.44608175313031,
 19.236726805015845,
 19.67934027424565,
 19.254288496794523]

In [38]:
final_train_loss_k4

[160.31795144081116,
 113.04050695896149,
 85.61802884737651,
 70.01726318299771,
 60.243804564476015,
 53.58560004432996,
 48.752012491226196,
 45.08027220219374,
 42.19868635204103,
 39.87309016823769,
 37.95896653803912,
 36.34588592549165,
 34.97396648021845,
 33.788192262819834,
 32.75368056456248,
 31.842112563550472,
 31.03094351712395,
 30.30598207844628,
 29.653420431049245,
 29.064938099384307]

In [39]:
def re_normalization(x, mean, std):
    x = x * std + mean
    return x


def max_min_normalization(x, _max, _min):
    x = 1. * (x - _min)/(_max - _min)
    x = x * 2. - 1.
    return x


def re_max_min_normalization(x, _max, _min):
    x = (x + 1.) / 2.
    x = 1. * x * (_max - _min) + _min
    return x


In [40]:
def predict_and_save_results_astgcn(net, data_loader, data_target_tensor, global_step, metric_method,_mean, _std, params_path, type):
    '''

    :param net: nn.Module
    :param data_loader: torch.utils.data.utils.DataLoader
    :param data_target_tensor: tensor
    :param epoch: int
    :param _mean: (1, 1, 3, 1)
    :param _std: (1, 1, 3, 1)
    :param params_path: the path for saving the results
    :return:
    '''
    net.train(False)  # ensure dropout layers are in test mode

    with torch.no_grad():

        data_target_tensor = data_target_tensor.cpu().numpy()

        loader_length = len(data_loader)  # nb of batch

        prediction = []  #storing the batch output

        input = []  #storing the batch input

        for batch_index, batch_data in enumerate(data_loader):

            encoder_inputs, labels = batch_data

            ##Taking only the single input feature
            input.append(encoder_inputs[:, :, 0:1].cpu().numpy())  # (batch, T', 1)

            outputs = net(encoder_inputs)

            prediction.append(outputs.detach().cpu().numpy())

            if batch_index % 100 == 0:
                print('predicting data set batch %s / %s' % (batch_index + 1, loader_length))

        input = np.concatenate(input, 0)

        input = re_normalization(input, _mean, _std)

        prediction = np.concatenate(prediction, 0)  # (batch, T', 1)

        print('input:', input.shape)
        print('prediction:', prediction.shape)
        print('data_target_tensor:', data_target_tensor.shape)
        output_filename = os.path.join(params_path, 'recent3pts04k4output_epoch_%s_%s' % (global_step, type))
        np.savez(output_filename, input=input, prediction=prediction, data_target_tensor=data_target_tensor)


        excel_list = []

        ###prediction length has the shape of feature of a certain node
        prediction_length = prediction.shape[2]

        for i in range(prediction_length):

            ### ensuring number of data samples in target sensor is same as that of prediction tensor
            assert data_target_tensor.shape[0] == prediction.shape[0]
            print('current epoch: %s, predict %s points' % (global_step, i))
            if metric_method == 'mask':

                ## calculating the value of the feature prediction of each node for T timesteps
                mae = masked_mae_test(data_target_tensor[:, :, i], prediction[:, :, i],0.0)
                rmse = masked_rmse_test(data_target_tensor[:, :, i], prediction[:, :, i],0.0)
                mape = masked_mape_np(data_target_tensor[:, :, i], prediction[:, :, i], 0)
            else :
                mae = mean_absolute_error(data_target_tensor[:, :, i], prediction[:, :, i])
                rmse = mean_squared_error(data_target_tensor[:, :, i], prediction[:, :, i]) ** 0.5
                mape = masked_mape_np(data_target_tensor[:, :, i], prediction[:, :, i], 0)
            print('MAE: %.2f' % (mae))
            print('RMSE: %.2f' % (rmse))
            print('MAPE: %.2f' % (mape))
            excel_list.extend([mae, rmse, mape])

        # print overall results
        if metric_method == 'mask':
            mae = masked_mae_test(data_target_tensor.reshape(-1, 1), prediction.reshape(-1, 1), 0.0)
            rmse = masked_rmse_test(data_target_tensor.reshape(-1, 1), prediction.reshape(-1, 1), 0.0)
            mape = masked_mape_np(data_target_tensor.reshape(-1, 1), prediction.reshape(-1, 1), 0)
        else :
            mae = mean_absolute_error(data_target_tensor.reshape(-1, 1), prediction.reshape(-1, 1))
            rmse = mean_squared_error(data_target_tensor.reshape(-1, 1), prediction.reshape(-1, 1)) ** 0.5
            mape = masked_mape_np(data_target_tensor.reshape(-1, 1), prediction.reshape(-1, 1), 0)
        print('all MAE: %.2f' % (mae))
        print('all RMSE: %.2f' % (rmse))
        print('all MAPE: %.2f' % (mape))
        excel_list.extend([mae, rmse, mape])
        print(excel_list)

In [41]:
def predict_main(global_step, data_loader, data_target_tensor,metric_method, _mean, _std, type):
    '''

    :param global_step: int
    :param data_loader: torch.utils.data.utils.DataLoader
    :param data_target_tensor: tensor
    :param mean: (1, 1, 3, 1)
    :param std: (1, 1, 3, 1)
    :param type: string
    :return:
    '''
    params_path = '/content/drive/MyDrive/COMP9491_ASTGCN_Model/Dataset_PEMS07/Best_Epoch_Saved_Recent/'
    params_filename = os.path.join(params_path, 'PEMS04_3pts_k4_epoch_%s.params' % global_step)
    print('load weight from:', params_filename)

    model_hour_k4.load_state_dict(torch.load(params_filename))

    predict_and_save_results_astgcn(model_hour_k4, data_loader, data_target_tensor, global_step, metric_method,_mean, _std, params_path, type)

###### Prediction for the Recent Model, when K = 4

In [42]:
predict_main(best_epoch_k4,test_loader, test_target_tensor,'unmask', _mean, _std, 'test' )

load weight from: /content/drive/MyDrive/COMP9491_ASTGCN_Model/Dataset_PEMS07/Best_Epoch_Saved_Recent/PEMS04_3pts_k4_epoch_17.params
predicting data set batch 1 / 54
input: (3396, 307, 1, 12)
prediction: (3396, 307, 3)
data_target_tensor: (3396, 307, 3)
current epoch: 17, predict 0 points
MAE: 19.18
RMSE: 30.66
MAPE: 0.13
current epoch: 17, predict 1 points
MAE: 19.64
RMSE: 31.37
MAPE: 0.14
current epoch: 17, predict 2 points
MAE: 20.21
RMSE: 32.19
MAPE: 0.14
all MAE: 19.68
all RMSE: 31.41
all MAPE: 0.14
[19.178371, 30.656719861333013, 0.134039, 19.641085, 31.372217062521557, 0.13765338, 20.207907, 32.190774636216624, 0.14109091, 19.675787, 31.412818032114693, 0.13759445]
